# BDD tmdb

In [1]:
# Importation des modules
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re

# Définir les options d'affichage pour afficher toutes les lignes sans troncature
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)

# Modifier les options d'affichage pour limiter les décimales
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
# Téléchargement de la BDD
tmdb = pd.read_csv("data/tmdb_full.csv")
 
# Affichage des info
print(tmdb.info())
print()
# Affichage des dimensions
print("Les dimensions sont",tmdb.shape)

tmdb.sample()

C:\Users\titgr\AppData\Local\Temp\ipykernel_29892\1429173555.py:2: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  tmdb = pd.read_csv("data/tmdb_full.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309572 entries, 0 to 309571
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   adult                         309572 non-null  bool   
 1   backdrop_path                 151760 non-null  object 
 2   budget                        309572 non-null  int64  
 3   genres                        309572 non-null  object 
 4   homepage                      44262 non-null   object 
 5   id                            309572 non-null  int64  
 6   imdb_id                       309572 non-null  object 
 7   original_language             309572 non-null  object 
 8   original_title                309572 non-null  object 
 9   overview                      282512 non-null  object 
 10  popularity                    309572 non-null  float64
 11  poster_path                   264159 non-null  object 
 12  production_countries          309572 non-nul

,adult,backdrop_path,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,production_companies_name,production_companies_country
220773,False,/jJJbqu5XQUxYrAdr397JGjhhIPe.jpg,0,[],NaN,128351,tt0199305,it,Aitanic,NaN,...,0,[],Released,NaN,Aitanic,False,4.10,4,[],NaN


In [ ]:
# # Compter le nombre de modalités des variables catégorielles et le % de \N
# my_colonne = ['adult', 'backdrop_path', 'budget', 'genres', 'homepage', 'id',
#        'imdb_id', 'original_language', 'original_title', 'overview',
#        'popularity', 'poster_path', 'production_countries', 'release_date',
#        'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
#        'video', 'vote_average', 'vote_count', 'production_companies_name',
#        'production_companies_country']

# # Dictionnaire pour stocker les résultats
# resultats = {}

# # Appliquer value_counts(normalize=True) * 100 pour chaque colonne
# for colonne in my_colonne:
#     resultats[colonne] = tmdb[colonne].value_counts(normalize=True) * 100

# # # Afficher les résultats
# # for colonne, valeurs in resultats.items():
# #     print(f"Pourcentage de chaque modalité pour la colonne '{colonne}':")
# #      print(valeurs)
# #     print()  # Séparateur pour plus de lisibilité

In [ ]:
# # Filtrer le DataFrame pour conserver uniquement les lignes où 'original_title' et 'title' sont différents
# filtered_df = tmdb[tmdb['original_title'] != tmdb['title']]

# # Afficher le DataFrame après le filtrage
# print("\nDataFrame après le filtrage (valeurs différentes dans 'original_title' et 'title'):")
filtered_df[['original_title', 'title']].sample(2)

In [ ]:
# Filtrer le DataFrame pour conserver uniquement les lignes où 'original_title' et 'title' sont égaux
filtered_df2 = tmdb[tmdb['original_title'] == tmdb['title']]

# Afficher le DataFrame après le filtrage
print("\nDataFrame après le filtrage (valeurs différentes dans 'original_title' et 'title'):")
filtered_df2[['original_title', 'title']].sample(2)

In [ ]:
# A revoir
# est ce qu'on remplace les lignes original_title par title ?

### Création du DF tmdb_clean

In [ ]:
# Création du dataframe 
tmdb_clean = pd.DataFrame(tmdb)

In [ ]:
# Modification des noms de colonnes conservées

    #Création du dictionnaire associant les anciens noms aux nouveaux noms de colonnes
dictionnaire_colonne_tmdb = {
    "id": "ID_TMDB",
    "imdb_id": "ID_title",
    "title": "title",
    "original_title": "original_title",
    "original_language": "original_language",
    "genres": "genres",
    "release_date": "release_year",
    "runtime": "runtime_minutes",
    "status": "status",
    "adult": "isAdult",
    "popularity": "popularity",
    "vote_average": "averageRating",
    "vote_count": "numVotes"
}

    #On renomme les variables grâce à la méthode rename
tmdb_clean = tmdb_clean.rename(dictionnaire_colonne_tmdb, axis = 1)

In [ ]:
tmdb_clean.info()

### Suppression des lignes 

#### Colonne status

In [ ]:
# Afficher les valeurs uniques de la colonne 'status'
tmdb_clean['status'].unique()

In [ ]:
# Retirer les lignes planned, canceled, rumored de la colonne 'status'
    # Créer la condition pour exclure les valeurs spécifiées
condition = (tmdb_clean['status'] != 'Planned') | (tmdb_clean['status'] != 'Canceled') | (tmdb_clean['status'] != 'Rumored')

    # Appliquer la condition pour filtrer le DataFrame
tmdb_clean = tmdb_clean[condition]

##### Colonne isAdult

In [ ]:
tmdb_clean['isAdult'].unique()

In [ ]:
# Retirer les lignes où la colonne 'isAdult' est True
tmdb_clean = tmdb_clean.loc[tmdb_clean['isAdult'] != True]

# Supprimer la colonne 'isAdult'
tmdb_clean = tmdb_clean.drop(columns= ['isAdult'], axis = 1)

In [ ]:
tmdb_clean.columns

#### Nettoyage des lignes pour ne conserver que des caractères latin

In [ ]:
# Fonction pour vérifier les caractères latins
def contains_only_latin_characters(x):
    if isinstance(x, str):
        # Cette expression régulière vérifie si tous les caractères de la chaîne 'x'
        # sont compris dans les plages Unicode des caractères ASCII et certains caractères latins étendus.
        return bool(re.match('^[\u0000-\u007F\u00A0-\u00FF]*$', x))
    return True  # Pour les cases qui ne sont pas des strings (e.g., NaN)

# Appliquer la fonction à chaque cellule et filtrer les lignes
filtered_rows = tmdb_clean.applymap(contains_only_latin_characters).all(axis=1)

# Filtrer le DataFrame pour garder uniquement les lignes où toutes les cellules sont validées
tmdb_clean = tmdb_clean[filtered_rows]

In [ ]:
# Fonction pour vérifier les caractères non latins
def contains_non_latin_characters(x):
    if isinstance(x, str):
        # Cette expression régulière recherche des caractères non latins
        return bool(re.search('[^\u0020-\u007E\u00A0-\u00FF]', x))
    return False  # Faux pour les cellules non-string comme int, float, NaN

# Liste pour stocker les noms des colonnes avec des caractères non latins
columns_with_non_latin = []

# Vérifier chaque colonne
for column in tmdb_clean.columns:
    if tmdb_clean[column].apply(contains_non_latin_characters).any():
        columns_with_non_latin.append(column)

# Afficher les noms des colonnes avec des caractères non latins
print("Colonnes avec des caractères non latins :", columns_with_non_latin)


In [ ]:
#tmdb_clean['overview'].value_counts()

In [ ]:
# Exemple d'application sur le DataFrame
tmdb_clean['production_companies_name'] = clean_column(tmdb_clean['production_companies_name'])
tmdb_clean['genres'] = clean_column(tmdb_clean['genres'])
tmdb_clean['spoken_languages'] = clean_column(tmdb_clean['spoken_languages'])

#### Gestion des valeurs manquantes

In [ ]:
# Détection des valeurs manquantes

    # On détecte les COLONNES contenant au moins une valeur manquante
print("le nombre de colonnes avec des valeurs manquantes est ", tmdb_clean.isna().any(axis = 0).sum())
print()
colonne_NA = tmdb_clean.isna().any(axis = 0)
print(colonne_NA)

# On détecte les LIGNES contenant au moins une valeur manquante
print("le nombre de lignes avec des valeurs manquantes est ", tmdb_clean.isna().any(axis = 1).sum())
ligne_NA = tmdb_clean.isna().any(axis = 1)

In [ ]:
# Vérifier les lignes vides dans le DataFrame
empty_rows = tmdb_clean.isnull().all(axis=1)

# Afficher les lignes vides
empty_rows_indices = empty_rows[empty_rows].index
print(empty_rows_indices)

Il n'y a pas de lignes vide dans le DF.

#### Gestion des NaN

In [ ]:
# Initialisation des comptes de NaN
nan_counts_per_column = {}
total_nan_counts = 0

# Boucle sur chaque colonne et comptage des NaN
for column in tmdb_clean.columns:
    nan_count = tmdb_clean[column].isna().sum()
    nan_counts_per_column[column] = nan_count
    total_nan_counts += nan_count

# Affichage des résultats
print("Nombre de NaN par colonne:")
for column, count in nan_counts_per_column.items():
    print(f"Colonne {column}: {count} NaN")

print("\nNombre total de NaN dans tout le DataFrame:")
print(total_nan_counts)

On ne retire pas les NaN pour le moment on verra au moment du merge s'il possible de recupèrer des données du df IMDB.

#### Nettoyer les colonnes production_companies_name, genres

In [ ]:
# Avant nettoyage
print(tmdb_clean['genres'].unique())
print()
print(tmdb_clean['production_companies_name'].unique())
print()
print(tmdb_clean['spoken_languages'].unique())


In [ ]:
# # nettoyer les colonnes production_companies_name, genres
# def clean_column(col):
#     # Assurer que col est une série de chaînes de caractères et gérer les NaNs
#     if isinstance(col, pd.Series):  # Vérifie si 'col' est une série
#         col = col.fillna('')  # Remplace NaN par une chaîne vide

#         # Remplace les caractères indésirables et divise
#         clean_split_col = col.str.replace(r"[\['\]]", "", regex=True).str.split(', ')

#         # Joint de nouveau les éléments de chaîne valides
#         return clean_split_col.apply(lambda x: ', '.join(val for val in x if isinstance(val, str)))

#     return col  # Dans le cas où 'col' n'était pas une série

In [ ]:
def clean_column(col):
    # Assurer que col est une série de chaînes de caractères et gérer les NaNs
    if isinstance(col, pd.Series):  # Vérifie si 'col' est une série
        col = col.fillna('')  # Remplace NaN par une chaîne vide
        
        # Vérifie si les éléments de la colonne sont des chaînes
        if col.dtype == 'object':  # Cela inclut des chaînes et des objets
            # Remplace les caractères indésirables et divise
            clean_split_col = col.str.replace(r"[\['\]]", "", regex=True).str.split(', ')
            
            # Joint de nouveau les éléments de chaîne valides
            return clean_split_col.apply(lambda x: ', '.join(val for val in x if isinstance(val, str)))
    
    return col  # Dans le cas où 'col' n'était pas une série ou ne contenait pas de chaînes


In [ ]:
# Appliquer la fonction clean_column à chaque colonne du DataFrame
tmdb_clean = tmdb_clean.apply(clean_column)

In [ ]:
# Application sur le DataFrame
tmdb_clean['production_companies_name'] = clean_column(tmdb_clean['production_companies_name'])
tmdb_clean['genres'] = clean_column(tmdb_clean['genres'])
tmdb_clean['spoken_languages'] = clean_column(tmdb_clean['spoken_languages'])

In [ ]:
# Créer une série avec les valeurs uniques et imprimer
# unique_genres = pd.Series(tmdb_clean['genres'].unique())
# print(unique_genres)

In [ ]:
# Retirer les espaces entre chaque valeur 
tmdb_clean['genres'] = tmdb_clean['genres'].str.replace(r',\s*', ',', regex=True)

In [ ]:
# # Créer une série avec les valeurs uniques et imprimer
# unique_genres = pd.Series(tmdb_clean['production_companies_name'].unique())
# print(unique_genres)

In [ ]:
# Retirer les espaces entre chaque valeur 
tmdb_clean['production_companies_name'] = tmdb_clean['production_companies_name'].str.replace(r',\s*', ',', regex=True)

In [ ]:
# Créer une série avec les valeurs uniques et imprimer
unique_genres = pd.Series(tmdb_clean['spoken_languages'].unique())
print(unique_genres)

In [ ]:
# Retirer les espaces entre chaque valeur 
tmdb_clean['spoken_languages'] = tmdb_clean['spoken_languages'].str.replace(r',\s*', ',', regex=True)

In [ ]:
# Créer une série avec les valeurs uniques et imprimer
unique_genres = pd.Series(tmdb_clean['spoken_languages'].unique())
print(unique_genres)

In [ ]:
# Après nettoyage
print(tmdb_clean['genres'].unique())
print()
print(tmdb_clean['production_companies_name'].unique())
print()
print(tmdb_clean['spoken_languages'].unique())

In [ ]:
# Remplacer les chaînes contenant uniquement des espaces par NaN dans tout le DataFrame
tmdb_clean = tmdb_clean.replace(r'^\s*$', np.nan, regex=True)

J'ai remplacé les chaînes contenant uniquement des espaces par NaN pour facilement les retrouver lors du merge final.

In [ ]:
print(tmdb_clean['spoken_languages'].unique())

##### Convertir la colonne 'release_Year' en datetime

In [ ]:
# Convertir les valeurs non-NA de la colonne 'release_Year' en type 'str', car il y a valeurs "float" dans la colonne
tmdb_clean['release_year'] = tmdb_clean['release_year'].astype(str)

# Extraire l'année à partir de la colonne 'release_Year' si elle n'est pas NaN
tmdb_clean['year'] = tmdb_clean["release_year"].apply(lambda x: x.split('-')[0] if not pd.isnull(x) else x).astype(int)

# Obtenir la valeur minimale et maximale de la colonne 'year' au lieu de 'release_Year'
min_year = tmdb_clean['year'].min()
max_year = tmdb_clean['year'].max()

# Afficher les résultats
print(f"Valeur minimale de 'year': {min_year}")
print(f"Valeur maximale de 'year': {max_year}")

In [ ]:
# Supprimer la colonne release_Year
tmdb_clean = tmdb_clean.drop('release_year', axis=1)

In [ ]:
# Retirer les lignes où 'year' est inférieur ou égale à 1985
tmdb_clean = tmdb_clean.loc[tmdb_clean['year'] >= 1985]

### Sélection des language ('FR', 'GB','IT', 'ES', 'DE')

In [ ]:
# Afficher le Décompte des Modalités de la Colonne "original_language"
print(tmdb_clean["original_language"].value_counts(normalize=True) * 100)

In [ ]:
##### optionnelle : A voir si on selectionne par la langue

In [ ]:
# Garder uniquement les lignes dont original_language est  'FR', 'GB','IT', 'ES', 'DE'
    # Définir les langues à garder
languages_to_keep = ['en', 'fr', 'es', 'de', 'it']

    # Filtrer le DataFrame pour garder uniquement les lignes avec les langues spécifiées
tmdb_clean = tmdb_clean.loc[tmdb_clean['original_language'].isin(languages_to_keep)]

#### Sélection des lignes qui ont des vote_count différent de 0.

In [ ]:
tmdb_clean = tmdb_clean[tmdb_clean['numVotes'] != 0]

In [ ]:
tmdb_clean.sample()
tmdb_clean.info()

In [ ]:
print(tmdb_clean.columns)

# Supprimer les colonnes non exploitables
tmdb_clean = tmdb_clean.drop(columns=["backdrop_path", "budget", "homepage", "id", "poster_path", 
                                       "production_countries", "revenue", "status", "tagline", 
                                       "video", "production_companies_country"])


## Vérification (doublons et naN)

In [ ]:
# Initialisation d'un dictionnaire pour stocker le nombre de doublons par colonne
duplicate_counts = {}

# Pour chaque colonne du DataFrame
for col in tmdb_clean.columns:
    # Utilisation de duplicated() pour détecter les doublons
    num_duplicates = tmdb_clean[col].duplicated(keep=False).sum() - tmdb_clean[col].duplicated(keep='first').sum()
    duplicate_counts[col] = num_duplicates

# Affichage du nombre de doublons par colonne
for col, count in duplicate_counts.items():
    print(f"Colonne '{col}' a {count} doublons")

In [ ]:
# Initialisation des comptes de NaN
nan_counts_per_column = {}
total_nan_counts = 0

# Boucle sur chaque colonne et comptage des NaN
for column in tmdb_clean.columns:
    nan_count = tmdb_clean[column].isna().sum()
    nan_counts_per_column[column] = nan_count
    total_nan_counts += nan_count

# Affichage des résultats
print("Nombre de NaN par colonne:")
for column, count in nan_counts_per_column.items():
    print(f"Colonne {column}: {count} NaN")

print("\nNombre total de NaN")

##### Optionnel

In [ ]:
# Selectionner et conserver les colonnes  qui nous intéressent pour le ML
columns_to_keep = ["ID_title", "popularity", "release_Year", "averageRating", "numVotes"]
tmdb_clean = tmdb_clean[columns_to_keep]

# 1-Sauvegarder en format CSV

In [ ]:
# 1-Sauvegarder en format CSV
tmdb_clean.to_csv('tmdb_clean.csv', index=False)